In [1]:
from util import vec
import pandas as pd
from nltk.corpus import stopwords
import numpy as np

In [2]:
#stop words are words that occur very frequently in the english language. Ex: of, the, a, an, etc
stop_words = set(stopwords.words('english'))

In [8]:
#data: is a dictionary that contains the label of the data as it's key and all data points under that label as elements of a list
#Ex: data['sum'] = ['print sum of points', 'show the addition of the points column', 'add points']
data = {}

for fn in os.listdir(os.path.join(os.getcwd(),'classes')):
    
    data[fn] = []
    _path = os.path.join('classes',fn)
    with open(_path) as f:
       data[fn] += f.readlines()

for key in data.keys():
    print(f'data[\'{key}\'] = {data[key][0:3]}...')

data['average'] = ['give me average of prize column\n', 'what is the average of prize\n', 'display the average of prize column\n']...
data['sum'] = ['give me sum of prize column\n', 'what is the sum of prize\n', 'display the sum of prize column\n']...
data['sort'] = ['sort prize column in ascending order\n', 'sort prize column in descending order\n', 'sort cost column in ascending order\n']...


In [15]:
#converts data to a df with one cloumn as the labels and the other column as data points
df = pd.DataFrame.from_dict(data, orient='index')
df = df.transpose().melt()

df = df.dropna()

df.variable.value_counts()

average    220
sum        220
sort        88
Name: variable, dtype: int64

In [16]:
#removing extra spaces (just an extra precaution)
df.value = df.value.str.strip()
df.head()

,variable,value
0,average,give me average of prize column
1,average,what is the average of prize
2,average,display the average of prize column
3,average,average of prize column
4,average,show the average of prize column


In [17]:
#A function that removes all stop words from a sentence.
#Input: "sum of points" Output: "sum points" 
def rm_stopwords(x):
    ans = ''
    for i in x.lower().split():
        if i not in stop_words:
            ans+=i+' '

    return ans

df.value = df.value.apply(rm_stopwords)
df.head()

,variable,value
0,average,give average prize column
1,average,average prize
2,average,display average prize column
3,average,average prize column
4,average,show average prize column


In [18]:
from sklearn.model_selection import train_test_split

#slitting the dataset into testing and training data
X_train, X_test, y_train, y_test = train_test_split( df.value, df.variable, test_size=0.1, random_state=3, stratify=df.variable)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(475,) (53,) (475,) (53,)


In [9]:
#taking the average of all word vectors present in a sentence
#Input: 5 50-dimension word embbeding Output: 1 50 dimension word embbeding
def sentenceVecAvg(x):
    a = vec(x)
    return sum(a)/len(a)

X_train = X_train.apply(sentenceVecAvg)
X_test = X_test.apply(sentenceVecAvg)

In [10]:
X_train = np.matrix(X_train.to_list())
X_test = np.matrix(X_test.to_list())

In [12]:
#indexing all labels

d={}
for index, function_name in enumerate(data.keys()):
    d[function_name] = index

{'average': 0, 'sum': 1, 'sort': 2}

In [13]:
#changing labels to index
y_train = y_train.apply(lambda x: d[x])
y_test = y_test.apply(lambda x: d[x])

In [14]:
y_train = np.array(y_train).reshape((-1,1))
y_test = np.array(y_test).reshape((-1,1))

In [15]:
from sklearn import svm

#training classifier
clf = svm.SVC()
clf.fit(X_train, y_train)

SVC()

In [16]:
#seeing accuracy
sum(clf.predict(X_test).reshape(y_test.shape)-y_test==0)/len(y_test)*100

array([100.])

In [18]:
#function that takes a sentence as input and gives the label as output.
def find_fn(ip):
    a = sentenceVecAvg(ip)
    ans = clf.predict(np.array(a).reshape((1,-1)))
    return ans
print(find_fn('sum age in data'))

[1]


In [19]:
clf.predict(sentenceVecAvg('sum data').reshape((1,50)))

array([1])

In [20]:
#saving the model
import pickle

#model name
filename = 'finalized_model.sav'

#model is saved under the models dir
_path = os.path.join('models',filename)
pickle.dump(clf, open(_path, 'wb'))

In [22]:
loaded_model = pickle.load(open(_path, 'rb'))
result = loaded_model.score(X_test, y_test)

In [29]:
loaded_model.predict(sentenceVecAvg('sort prize in order').reshape((1,50)))

array([2])